In [ ]:
import re

from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl

from path_fns.filepaths import (
    FINAL_CORRUPTED_OUTPUT_FILES_BASE,
    DEDUPE_OUTPUTS_FILES_BASE,
    TRANSFORMED_MASTER_DATA_ONE_ROW_PER_PERSON
)

from synthetic_data.prepare import (
    prepare_df
)

from synthetic_data.random_match import (
    get_prob_two_rnd_recs_are_match
)

import pandas as pd
import numpy as np
import os
from pathlib import Path

from graphs.linkagegraph import LinkageGraph

In [ ]:
dedupe_data_filename = "max_corruptions-50_prob_mult-1.0_set-1.parquet"

dedupe_data_path = os.path.join(
        DEDUPE_OUTPUTS_FILES_BASE, 
        dedupe_data_filename
    )

dedupe_records = pd.read_parquet(dedupe_data_path)
len(dedupe_records)


In [ ]:
dedupe_records.head(5)

In [ ]:
def calcMaxFmeasure(df, sample = 500000):
    df = df.sample(n=sample, axis=0, ignore_index=True).copy()
    match_probabilities = sorted(df.match_probability.unique())

    f_measures = []
    for match_probability in match_probabilities:
        true_positives = float(len(df.loc[(df["match_probability"] >= match_probability) & (df["cluster_l"] == df["cluster_r"])]))
        false_positives = float(len(df.loc[(df["match_probability"] >= match_probability) & (df["cluster_l"] != df["cluster_r"])]))
        false_negatives = float(len(df.loc[(df["match_probability"] < match_probability) & (df["cluster_l"] == df["cluster_r"])]))

        f_measures.append(true_positives / (true_positives + 0.5 * (false_positives + false_negatives)))

    max_fmeasure_index = f_measures.index(max(f_measures))

    return match_probabilities[max_fmeasure_index]

In [ ]:
# threshold = calcMaxFmeasure(dedupe_records)
threshold = 0.8951
print(threshold)

In [ ]:
len(dedupe_records.loc[(dedupe_records["match_probability"] >= threshold) & (dedupe_records["cluster_l"] != dedupe_records["cluster_r"])]) / len(dedupe_records)

In [ ]:
graphs_all = LinkageGraph(dedupe_records.sample(500000, axis=0, ignore_index=True), prob_threshold = threshold, min_order = 3)
df_all = graphs_all.get_measures()


In [ ]:
df_all.head(5)

In [ ]:
df_all[df_all["cluster_edge_betweenness_modularity"] > max(df_all["cluster_edge_betweenness_modularity"]) - 0.1]